In [1]:
import pandas as pd
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import cv2

from urllib import request
import urllib
from io import BytesIO
import numpy as np
from skimage import io
from tqdm import tqdm
import pickle

In [2]:
path = "./csv/침구-커튼-러그/침구"


In [3]:
df = pd.read_csv(path+".csv")
df.head()

,PROD_ID,PROD_NM,LV0,LV1,LV2,IMG_URL,LV0_onehot,LV1_onehot,LV2_onehot
0,366952,[위필] 천연라텍스 매트리스 [통몰드] 15cm 퀸(Q) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1,366953,[위필] 천연라텍스 매트리스 [통몰드] 15cm 슈퍼싱글(SS) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,367046,위필 천연라텍스 5T 미니 싱글 매트리스 오가닉커버 증정,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/367/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,366984,[위필] 천연라텍스 매트리스 [통몰드] 7.5cm 퀸 (Q) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
4,366985,[위필] 천연라텍스 매트리스 [통몰드] 7.5cm 슈퍼싱글 (SS) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [12]:
len(df)

14780

In [4]:
df["LV2"].unique()

array(['라텍스-메모리폼 매트', '방수커버-매트리스커버', '베개', '침구단품', '침구세트', '패드-스프레드',
       'S-SS(체크-스트라이프)', 'Q-D-K(체크-스트라이프)', 'Q-D-K(캐릭터-일러스트)',
       'Q-D-K(광목-린넨 내츄럴)', '매트리스커버', '베개-베캐커버', '베개커버(솜포함)', '이불커버',
       'S-SS(차렵)', 'Q-D-K(차렵)', 'Q-D-K(홑이불)', 'S-SS(홑이불)', '차렵이불',
       'Q-D-K(호텔-럭셔리)', '요-요커버', 'S-SS(무지-솔리드)', 'S-SS(일러스트-캐릭터)',
       'Q-D-K(무지-솔리드)', 'Q-D-K(러블리-플라워)', 'Q-D-K 극세사', '극세사이불', '혼수-예단침구',
       'Q-D-K(도트-패턴)', 'S-SS(극세사)', '요이불세트', '기능성 침구 기타', 'S-SS(러블리-플라워)',
       'S-SS(광목-린넨 내츄럴)', '베개커버(무지-솔리드)', '베개커버(프린팅-일러스트)', 'S-SS(도트-패턴)'],
      dtype=object)

In [5]:
for label in df["LV2"].unique():
    if not os.path.isdir(path):
        os.makedirs(path)

In [6]:
for idx in df.index:
    df["LV0"][idx] = df["LV0"][idx].replace("/","-")
    df["LV1"][idx] = df["LV1"][idx].replace("/","-")
    df["LV2"][idx] = df["LV2"][idx].replace("/","-")


ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
df.head()

,PROD_ID,PROD_NM,LV0,LV1,LV2,IMG_URL,LV0_onehot,LV1_onehot,LV2_onehot
0,366952,[위필] 천연라텍스 매트리스 [통몰드] 15cm 퀸(Q) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1,366953,[위필] 천연라텍스 매트리스 [통몰드] 15cm 슈퍼싱글(SS) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2,367046,위필 천연라텍스 5T 미니 싱글 매트리스 오가닉커버 증정,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/367/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,366984,[위필] 천연라텍스 매트리스 [통몰드] 7.5cm 퀸 (Q) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
4,366985,[위필] 천연라텍스 매트리스 [통몰드] 7.5cm 슈퍼싱글 (SS) 풀 세트,침구-커튼-러그,침구,라텍스-메모리폼 매트,https://image.hanssem.com/hsimg//gds/550/366/3...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [32]:
df[df["PROD_ID"]==255493]

,PROD_ID,PROD_NM,LV0,LV1,LV2,IMG_URL,LV0_onehot,LV1_onehot,LV2_onehot
2496,255493,오아스 차렵 이불 싱글 (Single),침구-커튼-러그,침구,Q-D-K(차렵),https://image.hanssem.com/hsimg//gds/550/255/2...,[0 0 0 0 0 0 0 1 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [8]:
df.to_csv(path+".csv", index=False, encoding="utf-8-sig")

In [81]:
# res = request.urlopen(df["IMG_URL"][0])
# img = Image.open(BytesIO(res.read()))

In [9]:
# dropped_indices = []
# imgs = []
# lv2_names = []
# for idx in tqdm(df.index):
#     try:
#         res = request.urlopen(df["IMG_URL"][idx])
#         img = Image.open(BytesIO(res.read())) 
#         if np.array(img).shape == (550,550,3) :
#             imgs.append(img)
#             lv2_name = df["LV2"][idx]
#             lv2_names.append(lv2_name)
#         else:
#             dropped_indices.append(idx)
        
#     except:
#         dropped_indices.append(idx)



100%|██████████| 3099/3099 [07:26<00:00,  6.94it/s]


In [8]:
dPath = "D:/download/csv/침구-커튼-러그/침구"

In [9]:
# with open(dPath+"/imgs.p", "wb") as f:
#     pickle.dump(imgs, f)

# with open(dPath+"/lv2_names.p", "wb") as f:
#     pickle.dump(lv2_names, f)

# with open(dPath+"/dropped_indices.p", "wb") as f:
#     pickle.dump(dropped_indices, f)

In [15]:
with open(dPath+"/dropped_indices.p", "rb") as f:
    dropped_indices = pickle.load(f)
len(dropped_indices)

35

In [11]:
with open(dPath+"/imgs.p", "rb") as f:
    imgs = pickle.load(f)
len(imgs)

14780

In [13]:
with open(dPath+"/lv2_names.p", "rb") as f:
    lv2_names = pickle.load(f)
len(lv2_names)

14780

In [14]:
df2 = df.copy()
if len(df2["IMG_URL"]) != len(imgs):
    df2 = df2.drop([df2.index[idx] for idx in dropped_indices])
    df2.to_csv(path+".csv", index=False, encoding="utf-8-sig")

In [21]:
df2[df["PROD_ID"]==255493]["IMG_URL"]

2496    https://image.hanssem.com/hsimg//gds/550/255/2...
Name: IMG_URL, dtype: object

In [23]:
for label in df["LV2"].unique():
    if not os.path.isdir(path+f"/{label}"):
        os.makedirs(path+f"/{label}")

In [24]:
for idx,img in tqdm(enumerate(imgs)):
    save_path = path+f"/{lv2_names[idx]}"
    # print(save_path)
    img.save(save_path+f"/{df['PROD_ID'][idx]}.png")
    
    


14780it [19:24, 12.69it/s]
